In [ ]:
!pip install torch
!pip install transformers
!pip install torchvision
!pip install langchain-huggingface
!pip install langchain-community
!pip install langchain-experimental
!pip install langchain-chroma
!pip install pypdf
!pip install sentence-transformers
!pip install chromadb
!pip install --upgrade chromadb posthog
!pip install bitsandbytes
!pip install langchain-classic
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Pre

In [ ]:
import torch
from transformers import BitsAndBytesConfig # Lượng tử hoá
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline # Load LLM model
from langchain_huggingface import HuggingFaceEmbeddings # Load Embedding
from langchain_huggingface.llms import HuggingFacePipeline # Load Embedding
from langchain_classic.memory import ConversationBufferMemory # Lưu chat history
from langchain_community.chat_message_histories import ChatMessageHistory # Lưu chat history
from langchain_community.document_loaders import PyPDFLoader # Xử lý file PDF
from langchain_classic.chains import ConversationalRetrievalChain # RAG chain
from langchain_experimental.text_splitter import SemanticChunker # Tách câu
from langchain_chroma import Chroma # Vector Database
from langchain_text_splitters import RecursiveCharacterTextSplitter # Tách câu
from langchain_core.runnables import RunnablePassthrough, RunnableParallel # Trả về Response
from langchain_core.output_parsers import StrOutputParser # Trả về Response
from langchain_classic import hub # Làm việc với các model từ HuggingFace

# **Trích xuất text từ file PDF**

In [ ]:
Loader = PyPDFLoader
FILE_PATH = "/content/drive/MyDrive/ML_DL_datasets/Object-Detection-with-YOLO (1).pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

In [ ]:
len(documents)

18

# **Khởi tạo Embedding model**

In [ ]:
embedding = HuggingFaceEmbeddings(
    model_name="bkai-foundation-models/vietnamese-bi-encoder"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [ ]:
semantic_splitter = SemanticChunker(
    embeddings=embedding,
    buffer_size=1,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    min_chunk_size=500,
    add_start_index=True
)

In [ ]:
docs = semantic_splitter.split_documents(documents)
print(f"Number of semantic chunks: {len(docs)}")

Number of semantic chunks: 18


# **Thiết lập Vector Database**

In [ ]:
vector_db = Chroma.from_documents(documents=docs, embedding=embedding)

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
result = retriever.invoke("YOLO là gì?")
print(f"Số lượng document phù hợp để trả lời câu hỏi: {len(result)}")

Số lượng document phù hợp để trả lời câu hỏi: 4


# **Load LLM model**

In [ ]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
MODEL_NAME = "lmsys/vicuna-13b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# **Thiết lập pipeline chạy Model**

In [ ]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=model_pipeline)

# **Hoàn thiện RAGFlow**

In [ ]:
# prompt = hub.pull("rlm/rag-prompt")
# prompt
from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_messages([
#     ("system", """
#     Bạn là một trợ lý AI thông minh và thân thiện, trả lời bằng tiếng Việt.
# Hãy sử dụng thông tin từ các tài liệu sau để trả lời câu hỏi một cách chính xác, ngắn gọn và tự nhiên nhất có thể.
# Nếu không biết thì hãy nói không biết, không bịa đặt.

# Context
# {context}
#     """),
#     ("human", "{question}")
# ])

prompt = ChatPromptTemplate.from_template(
    "Bạn là trợ lý AI và luôn trả lời bằng tiếng Việt.\n"
    "Dựa trên thông tin sau, hãy trả lời câu hỏi một cách chính xác và rõ ràng\n\n"
    "Context:\n{context}\n"
    "Câu hỏi:\n{question}\n"
    "Answer:"
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
USER_QUESTION = "Xin chào"
output = rag_chain.invoke(USER_QUESTION)

# Safely extract the answer based on the observed LLM output format
if "Answer:" in output:
    answer = output.split("Answer:")[1].strip()
elif "System:" in output:
    # Assuming the actual answer starts after the last 'System:' in a conversational output
    parts = output.rsplit("System:", 1)
    if len(parts) > 1:
        answer = parts[1].strip()
    else:
        answer = output.strip() # Fallback if no 'System:' is found
else:
    answer = output.strip() # If neither is found, take the whole output

print(answer)

Xin chào! Bạn muốn hỏi gì?


In [ ]:
answer = output.split("Answer:")
print(answer)

["System: \n    Bạn là một trợ lý AI thông minh và thân thiện, trả lời bằng tiếng Việt.\nHãy sử dụng thông tin từ các tài liệu sau để trả lời câu hỏi một cách chính xác, ngắn gọn và tự nhiên nhất có thể.\nNếu không biết thì hãy nói không biết, không bịa đặt.\n\nContext \n[Document(id='6779d767-c27b-4c40-a0fb-4632f0007a49', metadata={'total_pages': 18, 'creator': 'Google', 'creationdate': '', 'page': 13, 'start_index': 0, 'source': '/content/drive/MyDrive/ML_DL_datasets/Object-Detection-with-YOLO (1).pdf', 'page_label': '14', 'producer': 'PyPDF'}, page_content='Implementing YOLO'), Document(id='7ceb6b7b-85ea-44a4-ac0d-699cc3c65e33', metadata={'start_index': 0, 'producer': 'PyPDF', 'creator': 'Google', 'page_label': '6', 'total_pages': 18, 'page': 5, 'source': '/content/drive/MyDrive/ML_DL_datasets/Object-Detection-with-YOLO (1).pdf', 'creationdate': ''}, page_content='YOLO Algorithm'), Document(id='8142071c-ec09-4926-980f-469a058cb430', metadata={'source': '/content/drive/MyDrive/ML_DL_

In [ ]:
USER_QUESTION = "How are bounding boxes encoded?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split("Answer:")[1].strip()
print(answer)

Bounding boxes are encoded as coordinates of the bounding box's center, width (height) of the bounding box as a percentage of the cell's width (height), and a probability that the bounding box contains an object. The encoding is done by a CNN that predicts a vector y for each cell, which has 5B+C elements, where B is the number of bounding boxes predicted per cell and C is the number of classes.
